Initialization pof the IPC Communication 
1. Open OPENLCA, activate the relevent database
2. Tools > Developper Tools > IPC Server > run the server with the correct port

Help on the functions : https://greendelta.github.io/olca-ipc.py/olca/ipc.html#olca.ipc.Client.find

2 virtual environment :

-.venv : olca

-.test : olca_ipc

In [34]:
import pandas as pd
import numpy as np
import json
from tabulate import tabulate
import random as rand
import statistics as stats
import matplotlib as mp
import matplotlib.pyplot as plt
import scipy.stats as stat
#import olca as ipc
import olca_ipc as ipc
import olca_schema as o


from typing import Callable
###########################################################
client = ipc.Client(8080)

Finding a dataset and get Infos on it 

<font size=3px>Find a data set knowing it's type as well as one of the properties, useful to extract osme other properties such as the id , etc .... Finds only the first one, (if same name for example)</font>

In [ ]:
A = client.find(o.Process, "aluminium foil production")
print(A)
print(A.id)

Get  descritptors of a dataset : client.get_descriptors
return properties :  https://greendelta.github.io/olca-schema/classes/Ref.html

<font size=3px>This function is useful for exploring the content of a database. It returns a list of data set descriptors for a given data set type. A descriptor contains just a few information, like the name or category path, to understand the content of a data set. Descriptors are valid data set references; they can be used to reference a data set in a given context (e.g. referencing a process as calculation target in a calculation setup).</font>



In [ ]:
refs = client.get_descriptors(o.Process)
for ref in refs:
    print(ref.name, ': ',ref.id)


    


Get a full data set of a given type and ID : client.get https://greendelta.github.io/olca-schema/classes/RootEntity.html

In [ ]:
process = client.get(o.Process,"bd726e85-aaf9-4ec9-8617-8ededa87b89d")
print (tabulate(process.exchanges,headers="keys"))

Similarly Get a data set by a given type & a name 

<font size=3px>Note that the name does not have to be unique in an openLCA database, and in this case, it will just return the first entity from the database with the given name.</font>

In [ ]:
mass = client.get(o.FlowProperty, name="Mass")
print(mass)

Get all data sets of a given type using .get_all  , this produce a list object with each dataset of the datatype

<font size=3px>Returns all data sets of a given type from a database. This is not a practical method for all types of data sets and may is not available in a specific context. For example, it is fine to get all unit groups but a server would go down when you query for all processes in a large database. Typically, you just want to query all descriptors of a data set type.</font>

In [ ]:
allin = client.get_all(o.ProductSystem)
print(tabulate(allin,headers="keys"))

In [ ]:
### If you want to print only a specific attribute, since it is a list you need to print in a for loop
for group in client.get_all(o.ProductSystem):
    print(group.name)

Get the parameters of a data set 

Returns the (local) parameters of the specified data set. In case of processes and impact categories, a list of parameters is returned. For product systems, the respective parameter redefinitions are returned.

In [ ]:
parameters = client.get_parameters(o.ProductSystem, group.id)
for param in parameters:
    print(f"parameter: {param.name} = {param.value}", f"     Product System: {group.name}")
param = parameters[0]

In [ ]:
### In case the ID of the dataset is not known , one can read the list of all the datas and select the correct dataset using name or other parameters
Product_Syst_name = "sandwitch package production"
parameters = client.get_parameters(o.ProductSystem, client.find(o.ProductSystem,name=Product_Syst_name).id)
for param in parameters:
  print(f"parameter: {param.name} = {param.value}", f"     Product System: {Product_Syst_name}")


Get the providers of  technosphere flowos gives a flow ref & a provider process ref


<font size=3px>Technosphere flows are products or waste flows that can be linked in a product system model. For products, the providers are processes that produce this product as output. For waste flows, providers are waste treatment processes with an input of that waste flow. These provider and flow pairs can be linked to product inputs and waste outputs of other processes in a product system.</font>

In [ ]:
for p in client.get_providers():
    print (p.provider)
print(tabulate(client.get_providers(),headers="keys"))
print(len(client.get_providers()))

In [ ]:
### For a specific flow
providers = client.get_providers(
    flow=o.Ref(id="3102d53a-8544-4320-81b4-d295183f1597")
)
for p in providers:
    print(p)

Insert a data set // Needs the root entity

<font size=3px>Inserts or updates a provided data set in the database. This method is not available when a server runs in read-only mode.</font>

In [ ]:
ref = client.put(o.Source(name="Inter-process communication with openLCA",description="Test of updating a description from the API. Canyou update it ?"))
print(ref)


Update a data set // Needs the root entity

<font size=3px>To update a dataset, first step is to get the id of the dataset and use the same command. This method is not available when a server runs in read-only mode.</font>

In [ ]:
###### Properties (In this case, name) 
Process_name = "Inter-process communication with openLCA"
##### Be carful of the data set type
Updt = client.put(o.Source(id=client.get(o.Source, name=Process_name).id
                           ,name=Process_name,description="Test of updating a description from the API. Can you update it ?  Like really ?"))
#print(Updt)

Duplicate a data set with a new name// Needs the root entity

<font size=3px>To update a dataset, first step is to get the id of the dataset and use the same command. This method is not available when a server runs in read-only mode.</font>

In [ ]:
###### Properties (In this case, name) 
Duplicate_name = "aluminium foil production"
Duplicata = client.get(o.Process, name=Duplicate_name)
print(Duplicata.id)

In [ ]:
### Copy past the ID with a modification bellow
Duplicata.id = '08e41429-497d-47a0-b09f-7cc75b35d83c'
Duplicata.name = f"{Duplicate_name} _ Copy"
##### Be carful of the data set type
Duplicate = client.put(Duplicata)
print ("Created a copy of", Duplicata.name , "     with ID : ", Duplicata.id)

Delete a dataset // Needs the root entity

<font size=3px>Deletes the specified data set from the database. This method is not available when the server runs in read-only mode.</font>

In [ ]:
ref = client.delete(
    o.Ref(ref_type=o.RefType.Source, id =client.get(o.Source, name=Process_name).id)
)
print(ref)

Create a Product System // Needs the root entity

<font size=3px>This method creates a product system for a given process. It recursively links the processes of that system according to the given linking configuration.

Linking config : https://greendelta.github.io/olca-schema/classes/LinkingConfig.html </font>

In [2]:
process_ref = client.find(o.Process, "aluminium foil production")

config = o.LinkingConfig(
    prefer_unit_processes=True,
    provider_linking=o.ProviderLinking.PREFER_DEFAULTS,
)
system_ref = client.create_product_system(process_ref, config)
print (system_ref)
print(f"created product system {system_ref.name}, id={system_ref.id}")

Ref(id='28e1e90d-2edf-409e-9b21-bd74b0a60e44', category=None, description=None, flow_type=None, location=None, name='aluminium foil production', process_type=None, ref_unit=None, ref_type=<RefType.ProductSystem: 'ProductSystem'>)
created product system aluminium foil production, id=28e1e90d-2edf-409e-9b21-bd74b0a60e44


<font size=10px>Calculate Results</font>

<font size=3px>This method schedules a new calculation for a given setup. It directly returns a state object that contains the result ID (field @id). With this ID the the result state can be queried. </font>

In [89]:
### Check all the product system available
models = client.get_descriptors(o.ProductSystem)
for model in models:
    print(f"{model.name} :: {model.id}")

#### CHeck all the impact model available
methods = client.get_descriptors(o.ImpactMethod)
for method in methods:
    print(f"{method.name} :: {method.id}")


aluminium foil production :: 28e1e90d-2edf-409e-9b21-bd74b0a60e44
Testing :: 5b745732-bb1b-4425-adca-60088e9d9201


In [90]:
## Direct Selection 
model = client.find (o.ProductSystem, name = "aluminium foil production")

method = client.find(o.ImpactMethod, name="Testing")

In [92]:
# Select product Model & Impact assessment method 
print (model.name, "using the  assessment",method.name)

# create a calculation setup
setup = o.CalculationSetup(
    target= model,
    impact_method=method,
)

# run a calculation
client = ipc.Client()
result: ipc.Result = client.calculate(setup)


aluminium foil production using the  assessment Testing


In [93]:
### Check if Any error
state = result.get_state()
if state.error:
    print(f"calculation failed: {state.error}")
    exit(-1)


# or better do this:
state = result.wait_until_ready()
print(f"result id: {state.id}")
print(client)

result id: e9da60c5-a254-4d07-8951-62fe684bcd0e


In [94]:
A = result.get_total_impacts()
for i in A:
    assert i.impact_category
    print(f"{i.impact_category.name} {i.amount} {i.impact_category.ref_unit}")


Climate change 328.0 kg C02-eq


In [95]:
inventory = result.get_total_flows()
print(
    pd.DataFrame(
        data=[
            (
                i.envi_flow.flow.name,
                i.envi_flow.is_input,
                i.amount,
                i.envi_flow.flow.ref_unit,
            )
            for i in inventory
        ],
        columns=["Flow", "Is input?", "Amount", "Unit"],
    )
)


          Flow  Is input?  Amount Unit
0  solid waste      False   112.0   kg
1          CO2      False   328.0   kg
2    crude oil       True    25.5   kg
3      bauxite       True     5.0   kg


In [ ]:
# it is important to dispose a result when it is not needed anymore
result.dispose()

<font size=10px>Monte Carlo Simulations</font>

<font size=3px>Running a Monte Carlo Simulation is similar to a normal calculation. You first call the simulate method with a calculation setup. This schedules a first simulation run.  </font>

In [ ]:
import random as rand
import statistics as stats
import matplotlib as mp
import matplotlib.pyplot as plt
#####
client = ipc.Client(8080)


In [ ]:
models = client.get_descriptors(o.ProductSystem)
for model in models:
    print(f"{model.name} :: {model.id}")
model = models[0]

methods = client.get_descriptors(o.ImpactMethod)
for method in methods:
    print(f"{method.name} :: {method.id}")
method = methods[0]

In [ ]:
print("run iteration 1")
result = client.simulate(
    o.CalculationSetup(
        target=o.Ref(
        ref_type=o.RefType.ProductSystem,
        id=client.get(o.ProductSystem,name = "sandwitch package production").id,
        ),
    impact_method=o.Ref(method.id),
    )
)
### Check if Any error
state = result.get_state()
if state.error:
    print(f"calculation failed: {state.error}")
    exit(-1)

result.wait_until_ready()
print(f"result id: {state.id}")
print(client)
print("\n Results")

for i in result.get_total_impacts():
    assert i.impact_category
    print(f"{i.impact_category.name} {i.amount} {i.impact_category.ref_unit}   {i.impact_category.id}")

In [ ]:
# get the result for some indicator from the first iteration
indicator = client.find(o.ImpactCategory,name="Climate change, long term")
print (indicator)
indicator = client.get(o.ImpactCategory,"ca2d7af7-3e55-34f9-8ab7-b0c28ca7ddae")
print (indicator)
val = lambda: result.get_total_impact_value_of(indicator).amount
xs: list[float] = [val()]
print (xs)



In [ ]:
# collect the values from 999 more iterations
for i in range(0, 4999):
    print(f"run iteration {i+2}")
    result.simulate_next()
    result.wait_until_ready()
    xs.append(val())
print(xs)

In [ ]:
confidence_interval = stat.t.interval(0.95, len(xs) - 1, loc=np.mean(xs), scale=stats.stdev(xs))
#confidence_interval = stat.norm.interval(0.95, loc=np.mean(xs), scale=stat.sem(xs))
print("95% Confidence Interval:", confidence_interval,"Mean ",np.mean(xs))



plt.hist(xs,bins=150)
plt.axvline(x=confidence_interval[0], color='r', linestyle='--', label='Lower Bound')
plt.axvline(x=confidence_interval[1], color='r', linestyle='--', label='Upper Bound')

# Highlight the mean
plt.axvline(x=np.mean(xs), color='b', linestyle='-', label='Mean')

# Add labels and legend
plt.xlabel('kg of [CO2-eq]')
plt.ylabel('Data Points')
plt.legend()

# Show the plot
plt.show()


In [ ]:
confidence_interval = stat.t.interval(0.95, len(xs) - 1, loc=np.mean(xs), scale=stat.sem(xs))

print("95% Confidence Interval:", confidence_interval)

Results Investigation 

Get the techonsphere flow list using the .get_tech_flows function

Get the product system demand using .get_demand ()

Get all the technosphere flow consumed using the .get_total_requirements()  

In [96]:
tech_flows = result.get_tech_flows()
tech_requirements = result.get_total_requirements()
print(
    pd.DataFrame(
        [
            (tf.flow.id,tf.provider.name, tf.flow.name,treq.amount ,tf.flow.ref_unit)
             for tf, treq in zip(tech_flows,tech_requirements)   
        ],
        columns=["ID","Provider", "Flow","Value", "Unit"],
    ).head()
)
## Total demand of product system
A = result.get_demand()
print("Total Demand of product",A.amount)



                                     ID                   Provider  \
0  fcaf0889-2b64-41e8-9661-bb98856a7ea4  aluminium foil production   
1  707f112e-04cb-4c62-a4d6-282841d0bc41     electricity production   
2  788ab954-6bbe-41aa-add2-0ce003279a68       aluminium production   

             Flow  Value Unit  
0  aluminium foil    1.0   kg  
1     electricity   51.0   MJ  
2       aluminium    1.0   kg  
Total Demand of product 1.0


To get all the impacts for a specific technosphere flow : 


In [98]:
direct_flow_impacts = result.get_direct_impacts_of(tech_flow=tech_flows[1])
print(tabulate(direct_flow_impacts,headers="keys"),"\n")

singular_direct_flow_impact = result.get_direct_impact_of(impact_category=client.find(o.ImpactCategory,"Climate change"),tech_flow=tech_flows[1])
print(singular_direct_flow_impact.amount,singular_direct_flow_impact.impact_category.name)

  amount  impact_category
--------  ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
     153  Ref(id='8941a62f-edcd-4090-a719-eaeaeffd464a', category='Testing', description=None, flow_type=None, location=None, name='Climate change', process_type=None, ref_unit='kg C02-eq', ref_type=<RefType.ImpactCategory: 'ImpactCategory'>) 

153.0 Climate change


In [99]:
Ab = result.get_direct_impacts_of(tech_flow=tech_flows[2])
print(tabulate(Ab,headers="keys"))

  amount  impact_category
--------  ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
     120  Ref(id='8941a62f-edcd-4090-a719-eaeaeffd464a', category='Testing', description=None, flow_type=None, location=None, name='Climate change', process_type=None, ref_unit='kg C02-eq', ref_type=<RefType.ImpactCategory: 'ImpactCategory'>)


Investigate the Tech flows


In [100]:
#### Get the requirements for a specific tech flow _  The returned values are negative for inputs and positive for outputs of the respective flows. 
tech_flow_spec = result.get_scaled_tech_flows_of(tech_flows[2])
print(tabulate(tech_flow_spec,headers="keys"),"\n")

tech_flow_unscaled= result.get_unscaled_tech_flows_of(tech_flows[2])
print(tabulate(tech_flow_unscaled,headers="keys"),"\n")

scaling_fact = result.get_scaling_factors()
print(tabulate(scaling_fact,headers="keys"),"\n")


  amount  tech_flow
--------  -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
       0  TechFlow(flow=Ref(id='fcaf0889-2b64-41e8-9661-bb98856a7ea4', category=None, description=None, flow_type=<FlowType.PRODUCT_FLOW: 'PRODUCT_FLOW'>, location=None, name='aluminium foil', process_type=None, ref_unit='kg', ref_type=<RefType.Flow: 'Flow'>), provider=Ref(id='dcefef94-63ef-48cd-aebf-203c8c63511f', category=None, description=None, flow_type=<FlowType.PRODUCT_FLOW: 'PRODUCT_FLOW'>, location=None, na

Intervention Flows 
These are the flows that cross the boundary with the environment of the calculated system (this is why the short name is EnviFlow in the API). 

In [101]:
envi_flows = result.get_envi_flows()
tot_env_flows = result.get_total_flows ()
print(
    pd.DataFrame(
        [
            (ef.is_input, ef.flow.name, ef.flow.category,tot_ef.amount, ef.flow.ref_unit)
            for ef, tot_ef in zip(envi_flows, tot_env_flows)
        ],
        columns=["Is input?", "Flow", "Category","Amount", "Unit"],
    ).head()
)

specified_env_flow = result.get_total_flow_value_of(envi_flows[3])
print (specified_env_flow) #With amount and enfi_flow ref .

   Is input?         Flow Category  Amount Unit
0      False  solid waste     None   112.0   kg
1      False          CO2     None   328.0   kg
2       True    crude oil     None    25.5   kg
3       True      bauxite     None     5.0   kg
EnviFlowValue(amount=5.0, envi_flow=EnviFlow(flow=Ref(id='49a93207-858c-49ef-a4d3-d0b0576f803e', category=None, description=None, flow_type=<FlowType.ELEMENTARY_FLOW: 'ELEMENTARY_FLOW'>, location=None, name='bauxite', process_type=None, ref_unit='kg', ref_type=<RefType.Flow: 'Flow'>), is_input=True, location=None))


Get all the process contributions for a specific env. flows, for a specific impact 

In [102]:
env_flow_contrib = result.get_flow_contributions_of(envi_flow=envi_flows[3])
print(tabulate(env_flow_contrib,headers="keys"),"\n")

impact_flow_contrib = result.get_impact_contributions_of(impact_category=client.find(o.ImpactCategory,"Climate change"))
print(tabulate(impact_flow_contrib,headers="keys"))

  amount  tech_flow
--------  -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
       0  TechFlow(flow=Ref(id='fcaf0889-2b64-41e8-9661-bb98856a7ea4', category=None, description=None, flow_type=<FlowType.PRODUCT_FLOW: 'PRODUCT_FLOW'>, location=None, name='aluminium foil', process_type=None, ref_unit='kg', ref_type=<RefType.Flow: 'Flow'>), provider=Ref(id='dcefef94-63ef-48cd-aebf-203c8c63511f', category=None, description=None, flow_type=<FlowType.PRODUCT_FLOW: 'PRODUCT_FLOW'>, location=None, na

To get the direct biosphere interaction of a specific technosphere flow

In [103]:
tech_intensities = result.get_flow_intensities_of(tech_flow=tech_flows[0])
print(tabulate(tech_intensities,headers="keys"),"\n")

for x,env_flow in enumerate(envi_flows):
    tech_intensity = result.get_flow_intensity_of(envi_flow=envi_flows[x],tech_flow = tech_flows[0])
    print(round(tech_intensity.amount,6)," ",tech_intensity.envi_flow.flow.name)


  amount  envi_flow
--------  --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
   112    EnviFlow(flow=Ref(id='09d19276-507d-4489-8d38-5803312a6d46', category=None, description=None, flow_type=<FlowType.ELEMENTARY_FLOW: 'ELEMENTARY_FLOW'>, location=None, name='solid waste', process_type=None, ref_unit='kg', ref_type=<RefType.Flow: 'Flow'>), is_input=False, location=None)
   328    EnviFlow(flow=Ref(id='1692c65f-08e0-407a-8f30-fffc6da23ce9', category=None, description=None, flow_type=<FlowType.ELEMENTARY_FLOW: 'ELEMENTARY_FLOW'>, location=None, name='CO2', process_type=None, ref_unit='kg', ref_type=<RefType.Flow: 'Flow'>), is_input=False, location=None)
    25.5  EnviFlow(flow=Ref(id='7dc2ccdf-a066-44a3-8c68-99a95f6f59f4', category=None, description=None,

to get all the biosphère interactions of a specific techflows (including upstream)

In [104]:
total_tech_intensities= result.get_total_interventions_of(tech_flow=tech_flows[0])
print (tabulate(total_tech_intensities,headers="keys"),"\n")

for x,env_flow in enumerate(envi_flows):
    tech_total_intensity = result.get_total_intervention_of(envi_flow=envi_flows[x],tech_flow = tech_flows[0])
    print(round(tech_total_intensity.amount,6)," ",tech_total_intensity.envi_flow.flow.name)

  amount  envi_flow
--------  --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
   112    EnviFlow(flow=Ref(id='09d19276-507d-4489-8d38-5803312a6d46', category=None, description=None, flow_type=<FlowType.ELEMENTARY_FLOW: 'ELEMENTARY_FLOW'>, location=None, name='solid waste', process_type=None, ref_unit='kg', ref_type=<RefType.Flow: 'Flow'>), is_input=False, location=None)
   328    EnviFlow(flow=Ref(id='1692c65f-08e0-407a-8f30-fffc6da23ce9', category=None, description=None, flow_type=<FlowType.ELEMENTARY_FLOW: 'ELEMENTARY_FLOW'>, location=None, name='CO2', process_type=None, ref_unit='kg', ref_type=<RefType.Flow: 'Flow'>), is_input=False, location=None)
    25.5  EnviFlow(flow=Ref(id='7dc2ccdf-a066-44a3-8c68-99a95f6f59f4', category=None, description=None,

In [105]:
impact_categories = result.get_impact_categories()
print(
    pd.DataFrame(
        [(i.name, i.ref_unit) for i in impact_categories],
        columns=["Impact category", "Unit"],
    )
)

  Impact category       Unit
0  Climate change  kg C02-eq


In [106]:
import olca_ipc as ipc
import olca_schema as o


def main():

    # calculate a result
    client = ipc.Client(8080)
    setup = o.CalculationSetup(
        target=model
    )
    result = client.calculate(setup)
    result.wait_until_ready()

    # select the first best inventory flow and expand a tree for it
    flow = result.get_envi_flows()[0]
    print(f"upstream tree for {flow.flow.name} ({flow.flow.category})")
    expand(result, flow, [])

    # as always, dispose the result
    result.dispose()


def expand(r: ipc.IpcResult, flow: o.EnviFlow, path: list[o.TechFlow]):
    level = len(path)
    indent = "  " * level
    unit = flow.flow.ref_unit
    nodes = r.get_upstream_interventions_of(flow, path)
    for node in nodes:
        if node.result == 0 or not node.tech_flow:
            continue
        name = node.tech_flow.provider.name
        value = node.result
        print(f"{indent}- {value:.2E} {unit} :: {name}")

        # we stop the expansion after 3 levels; you can set other cut-offs like
        # result contributions etc.
        if level < 3:
            expand(r, flow, path + [node.tech_flow])
    pass


if __name__ == "__main__":
    main()


upstream tree for solid waste (None)
- 1.12E+02 kg :: aluminium foil production
  - 1.10E+02 kg :: aluminium production
    - 1.00E+02 kg :: electricity production
  - 2.00E+00 kg :: electricity production


In [107]:
import olca_ipc as ipc
import olca_schema as o


def main():

    # calculate a result
    client = ipc.Client(8080)
    setup = o.CalculationSetup(
        target=model,
        impact_method=method,
    )
    result = client.calculate(setup)
    result.wait_until_ready()

    # select the first best impact category and expand a tree for it
    indicator = client.find(o.ImpactCategory,"Climate change")
    print(f"upstream tree for {indicator.name}")
    expand(result, indicator, [])

    # as always, dispose the result
    result.dispose()


def expand(r: ipc.IpcResult, indicator: o.Ref, path: list[o.TechFlow]):
    level = len(path)
    indent = "  " * level
    unit = indicator.ref_unit
    nodes = r.get_upstream_impacts_of(impact_category=indicator, path=path)
    for node in nodes:
        if node.result == 0 or not node.tech_flow:
            continue
        name = node.tech_flow.provider.name
        value = node.result
        print(f"{indent}- {value:.2E} {unit} :: {name}")

        # we stop the expansion after 3 levels; you can set other cut-offs like
        # result contributions etc.
        if level < 3:
            expand(r, indicator, path + [node.tech_flow])
    pass


if __name__ == "__main__":
    main()

upstream tree for Climate change
- 3.28E+02 kg C02-eq :: aluminium foil production
  - 2.70E+02 kg C02-eq :: aluminium production
    - 1.50E+02 kg C02-eq :: electricity production
  - 3.00E+00 kg C02-eq :: electricity production


In [111]:
import olca_ipc as ipc
import olca_schema as o

imp_cat_for_sankey = client.find(o.ImpactCategory,"Climate change")
client = ipc.Client(8080)
# calculate a result for the default quantitative reference of a product system
result = client.calculate(
    o.CalculationSetup(
        target=model,
        impact_method=method,
    )
)
state = result.wait_until_ready()

# get the Sankey graph for an impact category
g = result.get_sankey_graph(
    o.SankeyRequest(
        impact_category=imp_cat_for_sankey,
        max_nodes=10,
    )
)
print(f"loaded a graph with {len(g.nodes)} nodes and {len(g.edges)} edges")
# prints something like: loaded a graph with 10 nodes and 14 edges
tot_scaling = result.get_total_impact_value_of(imp_cat_for_sankey)
ref = client.put(o.Result(id=state.id))
# finally, dispose the result
result.dispose()

print(tot_scaling.amount)


loaded a graph with 3 nodes and 3 edges
328.0


In [66]:
import plotly.graph_objects as go
print (tabulate(g.nodes,headers="keys"))

print (tabulate(g.edges,headers="keys"))

print (g.root_index)
A= []
nodes = []
for n in g.nodes : 
    nodes.append(dict(label=n.tech_flow.flow.name, pad=15))
    
print (nodes)



  direct_result    index  tech_flow                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  total_result
---------------  -------  --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [113]:
import plotly.graph_objects as go
import nbformat as nb

sorted_nodes = sorted(g.nodes, key=lambda n: n.index)
print (tabulate(sorted_nodes,headers="keys"))
# Define the nodes
nodes = []
for n in sorted_nodes:
    nodes.append(n.tech_flow.flow.name)

# Define the links
links = []
for e in g.edges : 
    links.append(dict(source=e.provider_index, target=e.node_index, value=e.upstream_share*tot_scaling.amount))


# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=250,
        thickness=20,
        line=dict(color='black', width=0.5),
        label=nodes,
    ),
    link=dict(
        source=[link['source'] for link in links],
        target=[link['target'] for link in links],
        value=[link['value'] for link in links],
    )
)])

fig.update_layout(title_text=f"Sankey Diagram of the {imp_cat_for_sankey.name} impacts for {model.name} process",  title_x=0.5)
# Display the plot
fig.show()


  direct_result    index  tech_flow                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  total_result
---------------  -------  --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------